In [17]:
from itertools import product
import pickle

import numpy as np

from algorithms.linear import LinearRegression
from algorithms.preprocessing import load_and_preprocess_dataset
import config

In [18]:
ridge_logs_dir = config.LOGS_DIR / 'ridge'
ridge_logs_dir.mkdir(exist_ok=True)

In [19]:
X, y, X_test, y_test = load_and_preprocess_dataset()

In [20]:
X = np.asarray(X)
y = np.asarray(y)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [21]:
alphas = [1e-2, 5e-3, 1e-3, 5e-4]

In [22]:
lambdas = [1e-2, 1e-3, 1e-4, 0]

In [23]:
batch_sizes = [32, 64]

In [24]:
indices = np.random.permutation(len(y))
fold_size = len(indices) // 5
scores = []
for alpha, lambda_, batch_size in product(alphas, lambdas, batch_sizes):
    model = LinearRegression(alpha, lambda_)
    fold_scores = []
    for fold in range(5):
        valid_indices = indices[fold * fold_size: (fold + 1) * fold_size]
        train_indices = indices[~np.isin(indices, valid_indices)]

        X_train, y_train = X[train_indices], y[train_indices]
        X_valid, y_valid = X[valid_indices], y[valid_indices]

        history = model.fit(X_train, y_train, X_valid, y_valid, epochs=500,
                            batch_size=batch_size, cold_start=True,
                            patience=50, min_delta=1e-3)
        fold_scores.append(history)
    with open(ridge_logs_dir / f'alpha_{alpha}-lambda_{lambda_}-batch_size_{batch_size}.pkl', 'wb') as f:
        pickle.dump(fold_scores, f)

alpha=0.0005, lambda=0, batch_size=64:  39%|███▉      | 196/500 [00:05<00:08, 34.67it/s, train_mse=0.2277384, valid_mse=0.2614349]
